In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

In [2]:
raw_data_csv = pd.read_csv("./../datasets/Index2018.csv")

In [3]:
df = raw_data_csv.copy()
df.head()

,date,spx,dax,ftse,nikkei
0,07/01/1994,469.90,2224.95,3445.98,18124.01
1,10/01/1994,475.27,2225.00,3440.58,18443.44
2,11/01/1994,474.13,2228.10,3413.77,18485.25
3,12/01/1994,474.17,2182.06,3372.02,18793.88
4,13/01/1994,472.47,2142.37,3360.01,18577.26


In [4]:
df.date.describe()

count           6269
unique          6269
top       07/01/1994
freq               1
Name: date, dtype: object

The top date value is misleading. It is not the highest date value. it is completely arbitrary. All the date values are 1, hence any single data point holds the top value. Lets convert them to date time.

In [5]:
df.date = pd.to_datetime(df.date, dayfirst=True)
df.head()

,date,spx,dax,ftse,nikkei
0,1994-01-07,469.90,2224.95,3445.98,18124.01
1,1994-01-10,475.27,2225.00,3440.58,18443.44
2,1994-01-11,474.13,2228.10,3413.77,18485.25
3,1994-01-12,474.17,2182.06,3372.02,18793.88
4,1994-01-13,472.47,2142.37,3360.01,18577.26


In [6]:
df.date.describe()

/tmp/ipykernel_12083/1371252037.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.date.describe()


count                    6269
unique                   6269
top       1994-01-07 00:00:00
freq                        1
first     1994-01-07 00:00:00
last      2018-01-29 00:00:00
Name: date, dtype: object

### Setting the datetime as new index

In [7]:
df.set_index("date", inplace=True)
df.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


### Setting the desired frequency 

Frequency cant be set 'h', 'd', 'w', 'm', hourly, daily, weekly, and monthly.

In [8]:
df = df.asfreq('d')
df.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-08,NaN,NaN,NaN,NaN
1994-01-09,NaN,NaN,NaN,NaN
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25


We can see that we have NaN values for 08, and 09 day, because we dont have observed data for those dates. This datset is closing price form stock exchange, i.e., we only have Monday - Friday data. If we look at the calendar, we can see 08, and 09 of 1995 Jan are weekends.

In [9]:
df = df.asfreq('b') # only business days 
df.head()

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.90,2224.95,3445.98,18124.01
1994-01-10,475.27,2225.00,3440.58,18443.44
1994-01-11,474.13,2228.10,3413.77,18485.25
1994-01-12,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


### Handling Missing Values 

In [10]:
df.isna().sum()

spx       8
dax       8
ftse      8
nikkei    8
dtype: int64

Initially, we had zero missing values. But, here we have 8 missing values. It was due to we set the frequency to business days.

In [11]:
df.spx = df.spx.fillna(method='ffill')

In [12]:
df.isna().sum()

spx       0
dax       8
ftse      8
nikkei    8
dtype: int64

In [13]:
df.ftse = df.ftse.fillna(method='bfill')

In [14]:
df.isna().sum()

spx       0
dax       8
ftse      0
nikkei    8
dtype: int64

In [15]:
df.dax = df.dax.fillna(value=df.dax.mean)

In [16]:
df.isna().sum()

spx       0
dax       0
ftse      0
nikkei    8
dtype: int64

In [17]:
df.nikkei = df.nikkei.fillna(method='ffill')

### Simplifying the dataset

we will analyze the S&P 500 data.

In [18]:
df['market_value'] = df.spx

In [19]:
df.describe()

,spx,ftse,nikkei,market_value
count,6277.000000,6277.000000,6277.000000,6277.000000
mean,1288.642547,5423.690398,14597.672753,1288.642547
std,487.868210,1145.568370,4043.795272,487.868210
min,438.920000,2876.600000,7054.980000,438.920000
25%,992.715221,4487.880000,10701.130000,992.715221
50%,1233.761241,5663.300000,15030.510000,1233.761241
75%,1460.250000,6304.630175,17860.470000,1460.250000
max,2872.867839,7778.637689,24124.150000,2872.867839


Lets delete the original data from df.

In [20]:
df.drop(["spx","ftse", "nikkei","dax"], axis=1, inplace=True)
df.describe()

,market_value
count,6277.000000
mean,1288.642547
std,487.868210
min,438.920000
25%,992.715221
50%,1233.761241
75%,1460.250000
max,2872.867839


### Splitting the time series data

We need to split the data into training and testing dataset. Most of the ML datsets are shuffled, but in case of time series data we have to mantain chronolgy of the data. we will use cutoff point to split the dataset.

dataset: _______________

train:   __________
test:              _____

Along the time domain. 

In [21]:
df.shape

(6277, 1)

In [22]:
train_size = int(df.shape[0] * 0.8)

In [23]:
df_train = df.iloc[:train_size]
df_test = df.iloc[train_size:]

In [24]:
df_train.tail(), df_test.tail()

(            market_value
 date                    
 2013-04-01   1562.173837
 2013-04-02   1570.252238
 2013-04-03   1553.686978
 2013-04-04   1559.979316
 2013-04-05   1553.278930,
             market_value
 date                    
 2018-01-23   2839.130362
 2018-01-24   2837.544008
 2018-01-25   2839.253031
 2018-01-26   2872.867839
 2018-01-29   2853.528411)